In [20]:
import unittest
from pydantic import BaseModel, Field, field_validator, model_validator
import plotly.graph_objects as go
import numpy as np
from typing import List, Optional
import uuid

uses a hexagonal coordinate system

ie, going clockwise around origin, pointy side up

| q | r | s |
 |----|--|--|
1 |-1 |0
1 |0 |-1
0 |1 |-1
-1| 1 |0
-1 |0 |1
0 |-1| 1

q is in the up right orientation  
r is rows in the down orientation  
s is in the up left direction  

two coordinates that are adjacant

https://www.redblobgames.com/grids/hexagons/

In [ ]:
class HexCoordinate(BaseModel):
    q: int
    r: int
    s: int
    @field_validator('s', mode='before')
    @classmethod
    def validate_cube_coordinates(cls, v, values):
        if values.data['q'] + values.data['r'] + v != 0:
            raise ValueError('Invalid cube coordinates')
        return v

    def get_adjacent_hexes(self):
        directions = [(1, -1, 0), (1, 0, -1), (0, 1, -1), (-1, 1, 0), (-1, 0, 1), (0, -1, 1)]
        adjacent = []
        for dq, dr, ds in directions:
            adjacent.append(HexCoordinate(q=self.q + dq, r=self.r + dr, s=self.s + ds))
        return adjacent

class GamePiece(BaseModel):
    hex_coordinates: Optional[HexCoordinate] = None
    icon: str = "�"
    team: str
    piece_id: int = Field(default_factory=lambda: str(uuid.uuid4()))
    location: str = Field(default='offboard') # 'offboard', 'board', 'stacked'

    @field_validator('team')
    @classmethod
    def validate_team(cls, v):
        if v not in ['black', 'white']:
            raise ValueError('Team must be either "black" or "white"')
        return v

class Spider(GamePiece):
    def __init__(self, hex_coordinates=None, team: str = 'white'):
        super().__init__(
            hex_coordinates=hex_coordinates, 
            team=team,
            icon="🕷️",
            location='offboard'
            )

class Ant(GamePiece):
    def __init__(self, hex_coordinates=None, team: str = 'white'):
        super().__init__(
            hex_coordinates=hex_coordinates, 
            team=team,
            icon="🐜",
            location='offboard'
            )

class Beetle(GamePiece):
    def __init__(self, hex_coordinates=None, team: str = 'white'):
        super().__init__(
            hex_coordinates=hex_coordinates, 
            team=team,
            icon="🪲",
            location='offboard'
            )

class Grasshopper(GamePiece):
    def __init__(self, hex_coordinates=None, team: str = 'white'):
        super().__init__(
            hex_coordinates=hex_coordinates, 
            team=team,
            icon="🦗",
            location='offboard'
            )

class QueenBee(GamePiece):
    def __init__(self, hex_coordinates=None, team: str = 'white'):
        super().__init__(
            hex_coordinates=hex_coordinates, 
            team=team,
            icon="🐝",
            location='offboard'
            )

class Ladybug(GamePiece):
    def __init__(self, hex_coordinates=None, team: str = 'white'):
        super().__init__(
            hex_coordinates=hex_coordinates, 
            team=team,
            icon="🐞",
            location='offboard'
            )

class Mosquito(GamePiece):
    def __init__(self, hex_coordinates=None, team: str = 'white'):
        super().__init__(
            hex_coordinates=hex_coordinates, 
            team=team,
            icon="🦟",
            location='offboard'
            )

class BoardState(BaseModel):
    pieces: dict = Field(default_factory=dict)

    def add_piece(self, piece_id: str, coordinates: HexCoordinate):
        for existing_piece in self.pieces.values():
            if existing_piece.hex_coordinates == coordinates:
                raise ValueError('Coordinates already occupied')
            if existing_piece.piece_id == piece_id:
                raise ValueError('Piece ID already exists on the board')

        self.pieces[piece_id] = GamePiece(hex_coordinates=coordinates)
        self.pieces[piece_id].location = 'board'

    def get_piece(self, piece_id: str):
        return self.pieces.get(piece_id, None)
    
class Player(BaseModel):
    name: str
    team: str
    pieces: List[GamePiece] = Field(default_factory=None)

    def __init__(self, name: str, team: str, pieces: Optional[List[GamePiece]] = None):
        if pieces is None:
            pieces = [
                *[Ant(team=team) for _ in range(12)],
                QueenBee(team=team)
                # super basic mode to start. add more pieces later
            ]
        super().__init__(name=name, team=team, pieces=pieces)

    @field_validator('team')
    @classmethod
    def validate_team(cls, v):
        if v not in ['black', 'white']:
            raise ValueError('Team must be either "black" or "white"')
        return v

class GameState(BaseModel):
    turn: int = Field(0, ge=0)
    white_player: Player = Player(name='white', team='white', pieces=[])
    black_player: Player = Player(name='black', team='black', pieces=[])
    current_team: str = Field(default='white')
    board_state: BoardState = Field(default_factory=BoardState)
    verbose: bool = Field(default=True)

    def __init__(self, white_player: Player = None, black_player: Player = None, **data):
        if white_player is None:
            white_player = Player(name='white', team='white')
        if black_player is None:
            black_player = Player(name='black', team='black')
        super().__init__(white_player=white_player, black_player=black_player, **data)

    def get_occupied_spaces(self):
        occupied = []
        for piece in self.board_state.pieces.values():
            if piece.location == 'offboard':    
                pass
            elif piece.location == 'board':
                occupied.append((piece.hex_coordinates.q, piece.hex_coordinates.r, piece.hex_coordinates.s))

        return occupied

    def get_available_spaces(self):
        if len(self.board_state.pieces) == 0:
            if self.verbose:
                print("No pieces on the board, returning center hex (0,0,0) as available space.")
            return HexCoordinate(q=0, r=0, s=0)  # If no pieces are on the board, return the center hex
        
        # get occupied spaces
        occupied = set(self.get_occupied_spaces())
        adjacent = set()

        for q, r, s in occupied:
            for dq, dr, ds in [(1, -1, 0), (1, 0, -1), (0, 1, -1), (-1, 1, 0), (-1, 0, 1), (0, -1, 1)]:
                adjacent.add((q + dq, r + dr, s + ds))
        relative = adjacent - occupied
        coords = []
        for coord in relative:
            coords.append(HexCoordinate(q=coord[0], r=coord[1], s=coord[2]))
        return coords

    def check_win_condition(self):
        # Check if either queen bee is completely surrounded
        white_queen = next((p for p in self.white_player.pieces if isinstance(p, QueenBee)), None)
        black_queen = next((p for p in self.black_player.pieces if isinstance(p, QueenBee)), None)
        if white_queen and white_queen.location == 'board':
            white_adjacent = set((hex.q, hex.r, hex.s) for hex in white_queen.hex_coordinates.get_adjacent_hexes())
            occupied = set(self.get_occupied_spaces())
            if white_adjacent.issubset(occupied):
                return 'black'  # Black wins
        if black_queen and black_queen.location == 'board':
            black_adjacent = set((hex.q, hex.r, hex.s) for hex in black_queen.hex_coordinates.get_adjacent_hexes())
            occupied = set(self.get_occupied_spaces())
            if black_adjacent.issubset(occupied):
                return 'white'  # White wins
        return None  # No winner yet

    @model_validator(mode='after')
    def validate_current_team(self):
        if self.current_team not in ['white', 'black']:
            raise ValueError('Current team must be either "white" or "black"')
        return self


class Turn(BaseModel):
    player: str
    piece_id: str
    action_type: str = Field(default_factory=lambda: ['move', 'place', 'forfeit'])
    target_coordinates: Optional[HexCoordinate] = None

    @staticmethod
    def validate_movement(turn, game_state):
        return turn
    
    @staticmethod
    def validate_placement(turn, game_state):
        

        # First piece must be placed at the center
        if game_state.turn == 0:
            if turn.target_coordinates != HexCoordinate(q=0, r=0, s=0):
                raise ValueError('First piece must be placed at the center (0,0,0)')
            else:
                return turn
        
        # check its next to an occupied space
        occupied = game_state.get_occupied_spaces()
        adjacent = turn.target_coordinates.get_adjacent_hexes()
        adjacent = [(hex.q, hex.r, hex.s) for hex in adjacent]
        occupied = set(occupied)
        adjacent = set(adjacent)
        if len(occupied.intersection(adjacent)) == 0:
            raise ValueError('Target coordinates must be adjacent to an occupied space')

        # check its not next to an opposite colour
        if game_state.turn > 1: # skip this check for the first placement
            player = game_state.white_player if turn.player == 'white' else game_state.black_player
            opponent = game_state.black_player if turn.player == 'white' else game_state.white_player
            player_piece_ids = [piece.piece_id for piece in player.pieces if piece.location == 'board']
            opponent_piece_ids = [piece.piece_id for piece in opponent.pieces if piece.location == 'board']
            player_occupied = [piece.hex_coordinates for pid, piece in game_state.board_state.pieces.items() if pid in player_piece_ids]
            opponent_occupied = [piece.hex_coordinates for pid, piece in game_state.board_state.pieces.items() if pid in opponent_piece_ids]
            player_adjacent = set()
            for hex in player_occupied:
                for adj in hex.get_adjacent_hexes():
                    player_adjacent.add((adj.q, adj.r, adj.s))
            opponent_adjacent = set()
            for hex in opponent_occupied:
                for adj in hex.get_adjacent_hexes():
                    opponent_adjacent.add((adj.q, adj.r, adj.s))
            target = (turn.target_coordinates.q, turn.target_coordinates.r, turn.target_coordinates.s)
            if target in opponent_adjacent:
                raise ValueError('Target coordinates cannot be adjacent to an opponent piece')
            # Check if the target coordinates are adjacent to an occupied space
            if turn.target_coordinates not in occupied:
                raise ValueError('Target coordinates must be adjacent to an occupied space')

        # check piece is offboard
        # get piece
        for p in game_state.board_state.pieces.values():
            if p.piece_id == turn.piece_id:
                piece = p
                break

        if piece is None:
            raise ValueError('Piece not found in player pieces')
        if piece.location != 'offboard':
            raise ValueError('Piece is already on the board')
        
        win = game_state.check_win_condition()
        return turn

class Game(BaseModel):
    game_state: GameState = Field(default_factory=GameState)
    history: List[Turn] = Field(default_factory=list)

    def apply_turn(self, turn: Turn):
        # Validate turn
        if turn.action_type == 'place':
            turn = Turn.validate_placement(turn, self.game_state)
            # actually place piece
            self.game_state.board_state.add_piece(turn.piece_id, turn.target_coordinates)

        elif turn.action_type == 'move':
            Turn.validate_movement(turn, self.game_state)
            # Movement logic to be implemented
            pass
        
        elif turn.action_type == 'forfeit':
            if self.game_state.verbose:
                print(f"{turn.player} has forfeited the game.")
            # Forfeit logic to be implemented
            pass
        
        else:
            raise ValueError('Invalid action type')
        
        # Update game state for next turn
        self.history.append(turn)
        self.game_state.turn += 1
        self.game_state.current_team = 'black' if self.game_state.current_team == 'white' else 'white'


In [ ]:
def hex_to_pixel(coord: HexCoordinate, size: float = 1.0):
    """Convert hex coordinate to pixel position for plotting."""
    x = size * (3/2 * coord.q)
    y = size * (np.sqrt(3)/2 * coord.q + np.sqrt(3) * coord.r)
    return x, y


def get_hexagon_vertices(x: float, y: float, size: float = 1.0):
    """Get vertices of a hexagon centered at (x, y)."""
    angles = np.linspace(0, 2*np.pi, 7)  # 7 points to close the hexagon
    vertices_x = x + size * np.cos(angles)
    vertices_y = y + size * np.sin(angles)
    return vertices_x, vertices_y

def visualize_game_board(board_state: BoardState, show_empty_hexes: Optional[List[HexCoordinate]] = None, show_coordinates: bool = True):
    """
    Visualize game pieces on hex coordinates
    """
    fig = go.Figure()
    
    hex_size = 0.95
    
    # CUSTOMIZATION: Team colors mapping
    team_colors = {
        "black": "#1D1A1A",  # black team
        "white": "#FFFFFF",  # white team
    }
    
    team_border_colors = {
        "black": "#000000",  # black border
        "white": "#808080",  # white border for visibility
    }
    
    # Draw empty hexes if provided
    if show_empty_hexes:
        for coord in show_empty_hexes:
            center_x, center_y = hex_to_pixel(coord, size=1.0)
            hex_x, hex_y = get_hexagon_vertices(center_x, center_y, hex_size)
            
            fig.add_trace(go.Scatter(
                x=hex_x,
                y=hex_y,
                fill='toself',
                fillcolor='#F5F5F5',
                line=dict(color='lightgray', width=1),
                mode='lines',
                showlegend=False,
                hoverinfo='text',
                hovertext=f'Empty: q={coord.q}, r={coord.r}, s={coord.s}',
            ))
    
    # Draw hexes with game pieces (only pieces that are on the board)
    for piece in board_state.pieces.values():
        # Skip pieces without coordinates (offboard pieces)
        if piece.hex_coordinates is None:
            continue
            
        # Skip pieces that are explicitly offboard
        if piece.location == 'offboard':
            continue
        
        coord = piece.hex_coordinates
        center_x, center_y = hex_to_pixel(coord, size=1.0)
        hex_x, hex_y = get_hexagon_vertices(center_x, center_y, hex_size)
        
        # Get team colors
        fill_color = team_colors.get(piece.team, 'lightgray')
        line_color = team_border_colors.get(piece.team, 'gray')
        
        # Draw hexagon
        fig.add_trace(go.Scatter(
            x=hex_x,
            y=hex_y,
            fill='toself',
            fillcolor=fill_color,
            line=dict(color=line_color, width=2),
            mode='lines',
            showlegend=False,
            hoverinfo='text',
            hovertext=f'{piece.__class__.__name__} ({piece.team})<br>Position: ({coord.q},{coord.r},{coord.s})',
        ))
        
        # Add piece icon
        fig.add_trace(go.Scatter(
            x=[center_x],
            y=[center_y],
            mode='text',
            text=[piece.icon],
            textfont=dict(size=60, color='black'),  # CUSTOMIZATION: Icon size
            showlegend=False,
            hoverinfo='skip'
        ))
        
        # Add coordinate labels (optional)
        if show_coordinates:
            fig.add_trace(go.Scatter(
                x=[center_x],
                y=[center_y - 0.3],
                mode='text',
                text=[f'({coord.q},{coord.r},{coord.s})'],
                textfont=dict(size=8, color='gray'),
                showlegend=False,
                hoverinfo='skip'
            ))

    fig.update_layout(
        title='Current State of the Game Board',
        xaxis=dict(
            scaleanchor='y',
            scaleratio=1,
            showgrid=True,
            zeroline=True,
            gridcolor='lightgray'
        ),
        yaxis=dict(
            showgrid=True,
            zeroline=True,
            gridcolor='lightgray'
        ),
        plot_bgcolor='white',
        width=800,
        height=800,
    )
    
    fig.show()


[HexCoordinate(q=1, r=-1, s=0),
 HexCoordinate(q=1, r=0, s=-1),
 HexCoordinate(q=0, r=1, s=-1),
 HexCoordinate(q=-1, r=1, s=0),
 HexCoordinate(q=-1, r=0, s=1),
 HexCoordinate(q=0, r=-1, s=1)]

In [ ]:
game = Game()

game.apply_turn(Turn(
    player='white', 
    piece_id=game.game_state.white_player.pieces[0].piece_id, 
    action_type='place', 
    target_coordinates=HexCoordinate(q=0, r=0, s=0)
    ))

game.apply_turn(Turn(
    player='black', 
    piece_id=game.game_state.black_player.pieces[0].piece_id, 
    action_type='place', 
    target_coordinates=HexCoordinate(q=1, r=-1, s=0)
    ))

# try a bad move
game.apply_turn(Turn(
    player='white', 
    piece_id=game.game_state.white_player.pieces[0].piece_id, 
    action_type='place', 
    target_coordinates=HexCoordinate(q=2, r=-2, s=0)
    ))

visualize_game_board(game.game_state.board_state, show_empty_hexes=game.game_state.get_available_spaces())


In [ ]:
game.get_available_spaces()

# add piece1 and piece2 to the board state for visualization
# psudo game moves
piece1 = game.white_player.pieces[0]
piece2 = game.black_player.pieces[4]

game.board_state.add_piece(piece1.piece_id, HexCoordinate(q=0, r=0, s=0))
game.board_state.add_piece(piece2.piece_id, HexCoordinate(q=1, r=-1, s=0))

available_spaces = game.get_available_spaces()


No pieces on the board, returning center hex (0,0,0) as available space.


In [ ]:
# attempt to add piece3 to an occupied space
piece3 = game.white_player.pieces[1]
try:
    game.board_state.add_piece(piece3.piece_id, HexCoordinate(q=0, r=0, s=0))
except ValueError as e:
    print(f"Error: {e}")  # Expected error since (0,0,0) is already occupied

Error: Coordinates already occupied


In [25]:
# attempt to add a peice that is already added
try:
    game.board_state.add_piece(piece1.piece_id, piece1, HexCoordinate(q=2, r=-2, s=0))
except ValueError as e:
    print(f"Error: {e}")  # Expected error since piece1 is already on the board

Error: Piece ID already exists on the board


adjacant

In [26]:
game.turn


0

In [51]:
def hex_to_pixel(coord: HexCoordinate, size: float = 1.0):
    """Convert hex coordinate to pixel position for plotting."""
    x = size * (3/2 * coord.q)
    y = size * (np.sqrt(3)/2 * coord.q + np.sqrt(3) * coord.r)
    return x, y


def get_hexagon_vertices(x: float, y: float, size: float = 1.0):
    """Get vertices of a hexagon centered at (x, y)."""
    angles = np.linspace(0, 2*np.pi, 7)  # 7 points to close the hexagon
    vertices_x = x + size * np.cos(angles)
    vertices_y = y + size * np.sin(angles)
    return vertices_x, vertices_y

def visualize_game_board(board_state: BoardState, show_empty_hexes: Optional[List[HexCoordinate]] = None, show_coordinates: bool = True):
    """
    Visualize game pieces on hex coordinates
    """
    fig = go.Figure()
    
    hex_size = 0.95
    
    # CUSTOMIZATION: Team colors mapping
    team_colors = {
        "black": "#1D1A1A",  # black team
        "white": "#FFFFFF",  # white team
    }
    
    team_border_colors = {
        "black": "#000000",  # black border
        "white": "#808080",  # white border for visibility
    }
    
    # Draw empty hexes if provided
    if show_empty_hexes:
        for coord in show_empty_hexes:
            center_x, center_y = hex_to_pixel(coord, size=1.0)
            hex_x, hex_y = get_hexagon_vertices(center_x, center_y, hex_size)
            
            fig.add_trace(go.Scatter(
                x=hex_x,
                y=hex_y,
                fill='toself',
                fillcolor='#F5F5F5',
                line=dict(color='lightgray', width=1),
                mode='lines',
                showlegend=False,
                hoverinfo='text',
                hovertext=f'Empty: q={coord.q}, r={coord.r}, s={coord.s}',
            ))
    
    # Draw hexes with game pieces (only pieces that are on the board)
    for piece in board_state.pieces.values():
        # Skip pieces without coordinates (offboard pieces)
        if piece.hex_coordinates is None:
            continue
            
        # Skip pieces that are explicitly offboard
        if piece.location == 'offboard':
            continue
        
        coord = piece.hex_coordinates
        center_x, center_y = hex_to_pixel(coord, size=1.0)
        hex_x, hex_y = get_hexagon_vertices(center_x, center_y, hex_size)
        
        # Get team colors
        fill_color = team_colors.get(piece.team, 'lightgray')
        line_color = team_border_colors.get(piece.team, 'gray')
        
        # Draw hexagon
        fig.add_trace(go.Scatter(
            x=hex_x,
            y=hex_y,
            fill='toself',
            fillcolor=fill_color,
            line=dict(color=line_color, width=2),
            mode='lines',
            showlegend=False,
            hoverinfo='text',
            hovertext=f'{piece.__class__.__name__} ({piece.team})<br>Position: ({coord.q},{coord.r},{coord.s})',
        ))
        
        # Add piece icon
        fig.add_trace(go.Scatter(
            x=[center_x],
            y=[center_y],
            mode='text',
            text=[piece.icon],
            textfont=dict(size=60, color='black'),  # CUSTOMIZATION: Icon size
            showlegend=False,
            hoverinfo='skip'
        ))
        
        # Add coordinate labels (optional)
        if show_coordinates:
            fig.add_trace(go.Scatter(
                x=[center_x],
                y=[center_y - 0.3],
                mode='text',
                text=[f'({coord.q},{coord.r},{coord.s})'],
                textfont=dict(size=8, color='gray'),
                showlegend=False,
                hoverinfo='skip'
            ))

    fig.update_layout(
        title='Current State of the Game Board',
        xaxis=dict(
            scaleanchor='y',
            scaleratio=1,
            showgrid=True,
            zeroline=True,
            gridcolor='lightgray'
        ),
        yaxis=dict(
            showgrid=True,
            zeroline=True,
            gridcolor='lightgray'
        ),
        plot_bgcolor='white',
        width=800,
        height=800,
    )
    
    fig.show()

# Usage
visualize_game_board(game.game_state.board_state, show_empty_hexes=game.game_state.get_available_spaces())
